<a href="https://colab.research.google.com/github/ToanToan110/CS114.K21/blob/master/DectectSarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df = pd.read_csv("chambiem1.csv")
df

,Column 1,1
0,Staples Center Employee Realizes He Left Shot ...,1
1,Entire Oklahoma State Team To Boycott Season A...,1
2,Fantasy Baseball League Commissioner Knows Han...,1
3,Adidas Unveils New Line Of Soccer Stuff,1
4,Red Sox Ask Fans To Switch From Racial To Homo...,1
...,...,...
994,Report: New ‘The Handmaid’s Tale’ Season Focus...,1
995,Cool Glitch Effect On Movie Studio Logo Must M...,1
996,Fans Excited As ‘Solo’ Trailer Sheds Light On ...,1
997,Archivists Discover Unpublished Michael Cricht...,1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ls

chambiem1.csv    drive/        Sarcasm_Headlines_Dataset.json
chinhthong1.csv  sample_data/  Sarcasm_Headlines_Dataset_v2.json


In [ ]:
!unzip 'drive/My Drive/detectsarcasm/news-headlines-dataset-for-sarcasm-detection.zip'

Archive:  drive/My Drive/detectsarcasm/news-headlines-dataset-for-sarcasm-detection.zip
replace Sarcasm_Headlines_Dataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df_1 = pd.read_json("Sarcasm_Headlines_Dataset.json",lines=True)

In [ ]:
df_2 = pd.read_csv("drive/My Drive/detectsarcasm/datatest.csv",)
df_2

In [ ]:
df_1

In [ ]:
df_1.info()
print("                  ")
df_2.info()

xem thông tin của biến df_1

In [ ]:
df_1=df_1.drop(labels="article_link",axis=1)

Xóa bỏ cột article link trong dataset


In [ ]:
import numpy as np
import string 
from string import digits
from string import punctuation
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
for i in df_1['headline']:
  i = str(i)

for a in df_2['Headline']:
  a = str(a)

tải về tệp tin giúp cho việc loại bỏ các thứ không quan trọng như dấu câu,...

Chạy vòng lập ép kiểu dữ liệu thành string để thao tác sau này tránh bị lỗi

In [ ]:
df_1

In [ ]:
hl_clean_1 = []
hl_clean_2 = []

for i in df_1['headline']:
    clean = i.translate(str.maketrans('', '', punctuation))
    clean = clean.translate(str.maketrans('', '', digits))
    hl_clean_1.append(clean)

for a in df_2['Headline']:
    clean = a.translate(str.maketrans('', '', punctuation))
    clean = clean.translate(str.maketrans('', '', digits))
    hl_clean_2.append(clean)


Dọn dẹp các chữ số và chấm câu

In [ ]:
hl_tokens_1 = []
hl_tokens_2 = []
for i in hl_clean_1:
  i = i.split()
  hl_tokens_1.append(i)
for i in hl_clean_2:
  i = i.split()
  hl_tokens_2.append(i)  


In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Init Lemmatizer 1
lemmatizer = WordNetLemmatizer()

hl_lemmatized_1 = []
for tokens in hl_tokens_1:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_1.append(lemm)

 # Init Lemmatizer 2 
hl_lemmatized_2 = []
for tokens in hl_tokens_2:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized_2.append(lemm)

Tách từ trong tập train và tập test để những từ trùng nhau chuyển về số


In [ ]:
hl_lemmatized_2

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_1)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_1)
X_train = pad_sequences(sequences, maxlen = max_token)
#padding giúp các chuỗi trong X_train có độ dài bằng nhau

####

max_features = 2000
max_token = len (hl_lemmatized_1)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized_2)
sequences = tokenizer.texts_to_sequences(hl_lemmatized_2)
X_test = pad_sequences(sequences, maxlen = max_token)
#padding giúp các chuỗi trong X_test có độ dài bằng nhau

In [ ]:
X_test

Vecto hóa 1 văn bản, giữ lại tối đa 2000 từ



In [ ]:
Y_train = df_1['is_sarcastic'].values
Y_train = np.vstack(Y_train)
#Nối 2 mảng bằng cách xếp chồng dọc, biến Y_train thành 1 ma trận
#X_trai,X_tes,Y_trai,Y_tes = train_test_split(X_train,Y_train,test_size=0.3, random_state = 42)
print (X_train)

#######
Y_test = df_2['Is_sarcastic'].values
Y_test = np.vstack(Y_test)
#biến Y_test thành 1 ma trận

In [ ]:
len(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf  =  RandomForestClassifier(n_estimators=300)
clf.fit(X_train, Y_train)

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [ ]:
le = LabelEncoder()
y_result=le.fit_transform(Y_test)
y_pred=clf.predict(X_test)

In [ ]:
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_result,y_pred))